In [ ]:
# Import libraries
%load_ext autoreload
%autoreload 2

from pulse_lib.interpolate_phase import interpolate_phase

from qick import *
import Pyro4
import pickle
%pylab inline

# Pyro communicates with board
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION=4

# Static IP proxy
ns_host = "192.168.2.99"
ns_port = 8888
proxy_name = "myqick"

# QICK object
ns = Pyro4.locateNS(host=ns_host, port=ns_port)
soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
print(soccfg)

In [ ]:
# Import phase calibration data
with open('pickles/calibrate_phase.pickle', 'rb') as handle:
    delta_phis = pickle.load(handle)

# Import SSB parameters
with open('pickles/ssb_params.pickle', 'rb') as handle:
    ssb_params = pickle.load(handle)

In [ ]:
# Config parameters
freq = 1010
phases = [ssb_params[freq][0], 0]
gains = [10000, ssb_params[freq][1]]

# Program DAC registers
prog = QickProgram(soccfg)
soc.reset_gens()
for ch in range(2):
    prog.declare_gen(ch=ch, nqz=1)
    prog.set_pulse_registers(ch = ch,
                             gain = gains[ch],
                             freq = prog.freq2reg(freq, gen_ch=ch),
                             phase = prog.deg2reg(phases[ch]+interpolate_phase(freq, delta_phis)[ch], gen_ch=ch),
                             style = "const",
                             length = prog.us2cycles(100, gen_ch=ch),
                             mode = "periodic",
                            )
    prog.pulse(ch=ch, t=0)

# Start DACs
prog.config_all(soc)
soc.tproc.start()

In [ ]:
# Stop DACs from generating waveforms
soc.reset_gens()